## XAIMINIROCKET


### Bloque 1
Carga las librerías usadas en todo el flujo y recarga el módulo minirocket_multivariate_variable por si se lo edita recientemente.

In [1]:
import importlib, os, json
import numpy as np
import pandas as pd
import minirocket_multivariate_variable as mmv
importlib.reload(mmv)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from collections import Counter
from tqdm import tqdm
import shap


/home/lgalarra/Documents/git/xai-for-minirocket/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Bloque 2
Define parámetros globales del experimento.

#### Entradas:

DATA_PATH → aquí se indica la ruta al juego de datos (archivo Parquet).
Ejemplos: "./data/mi_dataset.parquet" o "C:/proyecto/df.parquet".

LABEL_COL → nombre exacto de la columna de etiquetas en el parquet.

TEST_SIZE y RANDOM_STATE se pueden dejar por defecto.

#### Salidas: 
variables de configuración disponibles para los bloques siguientes.

In [2]:
DATA_PATH = "df_V.parquet"       
LABEL_COL = "RealDifficulty"
TEST_SIZE = 0.20
RANDOM_STATE = 42

df = pd.read_parquet(DATA_PATH)
print("Shape DF:", df.shape)


label_raw = (
    df[LABEL_COL].astype(str).str.strip().str.lower()
      .str.replace("í","i", regex=False)
      .str.replace("á","a", regex=False)
)

mapping_text2id = {"facil":0, "dificil":1}
unknown = sorted(set(label_raw.unique()) - set(mapping_text2id.keys()))
if unknown:
    raise ValueError(f"Valores inesperados en {LABEL_COL}: {unknown}")

y = label_raw.map(mapping_text2id).to_numpy(dtype=int)
print("Mapa etiqueta:", {"FACIL":0, "DIFICIL":1})


X2D = df.drop(columns=[LABEL_COL]).to_numpy(dtype=np.float32)
n, L = X2D.shape
X = X2D.reshape(n, 1, L)    # (n, C=1, L)
print("X:", X.shape, "| y:", y.shape)

cnt = Counter(y)
strat = y if min(cnt.values()) >= 2 else None
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=strat
)
print("X_train:", X_train.shape, "| X_test:", X_test.shape, "| clases:", np.unique(y))


Shape DF: (240, 7875)
Mapa etiqueta: {'FACIL': 0, 'DIFICIL': 1}
X: (240, 1, 7874) | y: (240,)
X_train: (192, 1, 7874) | X_test: (48, 1, 7874) | clases: [0 1]



### Bloque 3

Ajusta MiniRocket exclusivamente con X_train, aprendiendo dilataciones, sesgos y selección de canales. Estos parámetros quedan guardados internamente en mmv (estado global).

Transforma X_train a ϕ, devolviendo: Phi_train (n_train × F), la matriz de firmas PPV que se usará para entrenar la LR.

out_train["traces"], metadatos por firma (σ por tiempo, taps κ, dilatación, canales, sesgo), necesarios luego para la atribución temporal β(t).

Finalmente imprime la forma de Phi_train.

#### Entradas requeridas:

X_train con forma (n_train, C, L), preparado en bloques previos.

#### Salidas/efectos:

Estado interno de MiniRocket actualizado.

Phi_train listo para el bloque de LR(ϕ); traces disponibles para explicaciones.

In [3]:
mmv.fit_minirocket_parameters(X_train)
out_train = mmv.transform_prime(X_train)  
Phi_train = out_train["phi"]               
print("Phi_train:", Phi_train.shape)


Phi_train: (192, 9996)


In [4]:

clf_phi = make_pipeline(
    StandardScaler(with_mean=False),
    LogisticRegression(max_iter=2000, solver="lbfgs", multi_class="auto", n_jobs=-1)
).fit(Phi_train, y_train)
print("Clasificador LR(ϕ) entrenado.")

# 3.2) Conectar el clasificador para mmv.model_logit (vía ϕ)
mmv.set_phi_classifier_for_logits(clf_phi)
print("model_logit conectado (vía ϕ).")


Clasificador LR(ϕ) entrenado.
model_logit conectado (vía ϕ).


### Bloque 4

##### 1. Construye un background en el espacio ϕ (MiniRocket) a partir de TRAIN

Toma una muestra aleatoria de bg_size = min(128, len(X_train)) instancias de X_train (semilla fija para reproducibilidad).

Aplica mmv.transform_prime a cada instancia y apila sus firmas ϕ en phi_background (B × F).

Este background se usa para:

SHAP LinearExplainer (como referencia de distribución en ϕ).

Obtener la media en ϕ escalado: phi_bg_scaled = scaler.transform(phi_background), y su media mu_bg_scaled.

##### 2. Extrae el scaler y el clasificador lr desde el pipeline

scaler = clf_phi.named_steps["standardscaler"] permite escalar cualquier ϕ.

lr = clf_phi.named_steps["logisticregression"] expone los pesos/coeficientes del modelo en ϕ.

Calcula phi_bg_scaled y su media mu_bg_scaled (vector de tamaño F).

##### 3. Define helpers para baseline por medoide

_ensure_TC y _flatten_tc: aseguran y aplanan la forma temporal–canal para cálculos intermedios.

compute_medoids_by_class: para cada clase, selecciona el medoide (ejemplo más representativo) evaluando distancias en un subconjunto (máx. 300) para eficiencia.

pick_opposite_medoid: dada una instancia x_raw, predice su clase en ϕ y devuelve el medoide de la clase opuesta (baseline de referencia x̄), con fallback si falta.

In [5]:

rng = np.random.default_rng(42)
bg_size = min(128, len(X_train))
idx_bg = rng.choice(len(X_train), size=bg_size, replace=False)

phi_background = np.vstack([ mmv.transform_prime(X_train[k:k+1])["phi"]
                             for k in tqdm(idx_bg, desc="ϕ(background)") ])
print("phi_background:", phi_background.shape)

# 4.2) Extraer scaler y LR del pipeline
scaler = clf_phi.named_steps["standardscaler"]
lr     = clf_phi.named_steps["logisticregression"]

phi_bg_scaled = scaler.transform(phi_background)   # (B, F)
mu_bg_scaled  = phi_bg_scaled.mean(axis=0)         # media background en espacio escalado


ϕ(background): 100%|██████████| 128/128 [01:04<00:00,  1.99it/s]

phi_background: (128, 9996)


In [6]:
def _ensure_TC(x):
    x = np.asarray(x)
    if x.ndim == 3:
        assert x.shape[0] == 1
        x = x[0]
    if x.ndim == 2:
        return x.T if x.shape[0] < x.shape[1] else x
    raise ValueError(f"Forma no soportada: {x.shape}")

def _flatten_tc(x_tc):
    return _ensure_TC(x_tc).reshape(-1)

def compute_medoids_by_class(X_src, y_src, max_candidates=300, seed=42):
    rng = np.random.default_rng(seed)
    med = {}
    for cls in np.unique(y_src):
        Xc = X_src[y_src == cls]
        m = len(Xc)
        if m == 0:
            continue
        idx = np.arange(m)
        if m > max_candidates:
            idx = rng.choice(m, size=max_candidates, replace=False)
        flat = np.vstack([_flatten_tc(Xc[i:i+1][0].T) for i in idx]).astype(np.float32)
        norms = (flat**2).sum(1, keepdims=True)
        D2 = norms + norms.T - 2.0 * flat @ flat.T
        np.maximum(D2, 0.0, out=D2)
        D = np.sqrt(D2, dtype=np.float32)
        best_local = int(np.argmin(D.mean(1)))
        med[int(cls)] = Xc[idx[best_local]:idx[best_local]+1]
    return med

def pick_opposite_medoid(x_raw, clf_phi, medoids, transform_prime):
    phi_x = transform_prime(x_raw)["phi"]
    proba = clf_phi.predict_proba(phi_x)[0]
    clf_classes = clf_phi.classes_ if hasattr(clf_phi, "classes_") else np.arange(len(proba))
    order = np.argsort(-proba)
    y_hat_cls = int(clf_classes[order[0]])
    # Devuelve el medoid de la clase más probable NO elegida
    for j in order[1:]:
        cand = int(clf_classes[j])
        if cand in medoids:
            return medoids[cand], cand
    # fallback
    for cand in medoids:
        if cand != y_hat_cls:
            return medoids[cand], cand
    raise RuntimeError("No hay medoid opuesto disponible.")


### Bloque 5

Se alcula, en el conjunto bas, el medoide por clase (ejemplo más representativo).

Selecciona una instancia de prueba i para explicar.

Define la referencia (baseline) x̄ como el medoid de la clase opuesta a la predicción del modelo en ϕ para esa instancia.

Transforma x y x̄ al espacio de firmas ϕ y obtiene sus trazas (σ por tiempo, kernel κ, dilatación, etc.), necesarias para atribución temporal.

Convierte ambas series a forma temporal–canal (T, C) y calcula los logits f(x) y f(x̄), obteniendo Δf = f(x) − f(x̄), la cantidad total que deben explicar las atribuciones β(t).

##### Entradas requeridas:

X_train, y_train, X_test, y_test ya construidos; clf_phi entrenado y conectado con mmv.set_phi_classifier_for_logits(clf_phi); MiniRocket ajustado previamente.

##### Salidas principales:

X0_op (baseline por medoid opuesto) y su clase cls_opp.

phi_x, phi_x0 y sus traces, traces0.

fx, fx0 y delta_f (escala logit), que luego se usará para verificar cierre local (Σβ ≈ Δf).

In [7]:
X_base, y_base = X_train, y_train
medoids = compute_medoids_by_class(X_base, y_base, max_candidates=300)

i = 0
x_raw  = X_test[i:i+1]       
y_true = int(y_test[i])

X0_op, cls_opp = pick_opposite_medoid(x_raw, clf_phi, medoids, mmv.transform_prime)
print(f"[inst {i}] y_true={y_true}  |  baseline=medoid clase opuesta -> {cls_opp}")


[inst 0] y_true=0  |  baseline=medoid clase opuesta -> 1


In [8]:
out_x   = mmv.transform_prime(x_raw)
out_x0  = mmv.transform_prime(X0_op)
phi_x   = out_x["phi"]
phi_x0  = out_x0["phi"]
traces  = out_x["traces"]   
traces0 = out_x0["traces"]  

x_tc  = x_raw[0].T    # (T, C)
x0_tc = X0_op[0].T
fx  = float(mmv.model_logit(x_tc))
fx0 = float(mmv.model_logit(x0_tc))
delta_f = fx - fx0
print(f"fx={fx:.6f}  fx0={fx0:.6f}  Δf={delta_f:.6f}")


fx=0.196147  fx0=4.060584  Δf=-3.864437


### Bloque 6

Se estandariza las firmas ϕ de la instancia objetivo y de su referencia (baseline) usando el scaler del pipeline, para trabajar en el mismo espacio donde fue entrenada la LR. Luego identifica la clase objetivo (la que el modelo predice para la instancia) y toma el vector de pesos w_c de la regresión logística asociado a esa clase. Con eso calcula los α “brutos” en ϕ escalado mediante la forma cerrada de la LR: cada α_k mide la contribución del feature k como diferencia respecto a la media del background (μ en ϕ escalado) ponderada por w_c. A continuación calibra esos α para que suman exactamente Δf (la diferencia de logits entre la instancia y su baseline), garantizando local accuracy en el espacio de firmas. Después extrae la compuerta de referencia σ̄ por feature a partir de las traces del baseline y construye Δσ = σ − σ̄, un término ternario en {−1, 0, 1} que indica, para cada sello temporal y firma, si la activación cambió, no cambió o cambió en sentido opuesto entre la instancia y la referencia. Con los α calibrados y Δσ, propaga las contribuciones al tiempo mediante el operador de atribución: reparte cada α_k sobre los tiempos donde hay Δσ≠0, ponderando por los taps del kernel (patrón fijo 3×+2 y 6×−1) y por la dilatación

In [9]:
phi_x_scaled  = scaler.transform(phi_x)
phi_x0_scaled = scaler.transform(phi_x0)

proba   = clf_phi.predict_proba(phi_x)[0]
cls_idx = int(np.argmax(proba))
W = lr.coef_                  # (n_classes, F) o (1, F)
if W.ndim == 2 and W.shape[0] > 1:
    w_c = W[cls_idx]
else:
    w = W.ravel()
    w_c = w if cls_idx == 1 else -w

alphas_raw = (phi_x_scaled[0] - mu_bg_scaled) * w_c

alphas = mmv.calibrate_alphas_to_delta_f(alphas_raw, delta_f, phi_x_scaled, phi_x0_scaled)
print("∑α:", float(np.sum(alphas)), " | Δf:", float(delta_f))


∑α: -3.864436809354663  | Δf: -3.8644368093546633


In [10]:

sigma_ref = [tr0["sigma"] for tr0 in traces0]   

beta = mmv.propagate_luis(alphas, traces, x_tc, x0_tc, sigma_ref=sigma_ref, mode="channel_energy")
print("β shape:", beta.shape, " Σβ:", float(beta.sum()), " error:", float(beta.sum()-delta_f))


β shape: (7874, 1)  Σβ: -3.864436809354662  error: 1.3322676295501878e-15


In [11]:

err = float(beta.sum() - delta_f)
print(f"[Local accuracy] Δf={delta_f:.12g}  Σβ={beta.sum():.12g}  error={err:.2e}")

for k in [0, 1, 2, 123, 456, 999]:
    kappa = out_x["traces"][k]["kernel"]
    vals, counts = np.unique(kappa, return_counts=True)
    print(f"k={k} taps ->", dict(zip(vals, counts)))
print("phi_x.shape:", phi_x.shape, "| len(traces):", len(traces))


ok_delta_sigma = all(set(np.unique(tr["sigma"] - tr0["sigma"])) <= {-1,0,1}
                     for tr,tr0 in zip(traces, traces0))
print("Δσ en {-1,0,1}:", ok_delta_sigma)


[Local accuracy] Δf=-3.86443680935  Σβ=-3.86443680935  error=1.33e-15
k=0 taps -> {-1.0: 6, 2.0: 3}
k=1 taps -> {-1.0: 6, 2.0: 3}
k=2 taps -> {-1.0: 6, 2.0: 3}
k=123 taps -> {-1.0: 6, 2.0: 3}
k=456 taps -> {-1.0: 6, 2.0: 3}
k=999 taps -> {-1.0: 6, 2.0: 3}
phi_x.shape: (1, 9996) | len(traces): 9996
Δσ en {-1,0,1}: True


### Bloque 7

Este bloque construye un pipeline completo para cargar datos, entrenar MiniRocket + LR(ϕ) y, para un conjunto elegido (TEST o ALL), exportar:

las atribuciones temporales β(t) por instancia,

los α de SHAP en ϕ (tanto raw como calibrados a Δf),

y un CSV de métricas de cierre (∑β vs Δf).

A continuación, qué hace cada sección:

#### 1. Configuración
Define parámetros editables:

DATA_PATH: ruta del juego de datos (Parquet). Aquí debe apuntar Luis a su archivo.

LABEL_COL: nombre de la columna de etiqueta en el Parquet.

RUN_SCOPE: qué dividir exportar ("test" o "all").

BASELINE_FROM: de dónde tomar los medoides para la baseline ("train" o "all").

OUT_DIR: carpeta donde se guardan los artefactos.

#### 2. Carga y preparación de datos
Lee el Parquet (DATA_PATH), normaliza la etiqueta FACIL/DIFICIL → 0/1, construye X con forma (n, 1, L) (todas las columnas excepto la etiqueta) y parte en X_train/X_test (estratificado cuando es posible).

#### 3. MiniRocket + LR(ϕ)

Ajusta los parámetros de MiniRocket solo con TRAIN.

Transforma X_train a ϕ (Phi_train).

Entrena una Logistic Regression sobre ϕ y la conecta para poder pedir logits desde la serie cruda.

#### 4. Background para SHAP en ϕ escalado
Muestrea un subconjunto de TRAIN, lo transforma a ϕ, lo escala con el scaler del pipeline y crea phi_bg_scaled, que se usa como background del explicador SHAP. (También se obtiene scaler y lr del pipeline.)

#### 5. Explainer SHAP
Crea un LinearExplainer sobre la LR usando phi_bg_scaled como background.

#### 6. Helpers y baseline por medoide opuesto
Define utilidades para:

calcular medoides por clase (ejemplo más representativo),

y elegir, para cada instancia, como baseline el medoid de la clase opuesta a la predicción del modelo en ϕ.

#### 7. Fuente de medoids
Construye los medoids desde TRAIN o ALL según BASELINE_FROM.

#### 8. Selección del split a exportar
Decide si iterar sobre TEST o ALL según RUN_SCOPE. Ajusta una etiqueta split_tag para nombrar los archivos de salida.

#### 9. Preasignación de archivos de salida (memmap)
Detecta dimensiones:

F (nº de firmas ϕ),

T (longitud temporal),

N (nº de instancias a exportar),
y preasigna tres archivos memmap (eficientes en RAM):

betas_<split>.mmap → β(t) de tamaño (N, T),

alphas_shap_raw_<split>.mmap → α SHAP raw (N, F),

alphas_shap_cal_<split>.mmap → α SHAP calibrados (N, F).

#### 10 Bucle principal por instancia (exportación)
Para cada instancia del split elegido:

Selecciona la baseline = medoid opuesto.

Calcula Δf = logit(x) − logit(x̄).

Obtiene ϕ y traces de x y x̄; escala ϕ.

Calcula α SHAP en ϕ escalado con el explainer.

Calibra α SHAP para que ∑α = Δf (local accuracy en ϕ).

Propaga los α calibrados al tiempo con propagate_luis (usa Δσ ∈ {−1,0,1}, taps κ y dilatación) para obtener β(t) cumpliendo ∑β = Δf.

Guarda β(t), α SHAP raw y α SHAP calibrados en los memmaps, y añade una fila de resumen (Δf, ∑α, ∑β, error, clase del medoid usado).

#### 11. Cierre y resumen
Sincroniza (cierra) los memmaps y escribe efficiency_<split>.csv con métricas por instancia, incluida rel_error = |Σβ − Δf| / |Δf|. Imprime un resumen de los archivos generados.

In [17]:



DATA_PATH     = "df_V.parquet"
LABEL_COL     = "RealDifficulty"
TEST_SIZE     = 0.20
RANDOM_STATE  = 42

RUN_SCOPE     = "all"    # "test" | "all"
BASELINE_FROM = "all"   # "train" | "all"
OUT_DIR       = "method1_outputs"
os.makedirs(OUT_DIR, exist_ok=True)


df = pd.read_parquet(DATA_PATH)
label_raw = (df[LABEL_COL].astype(str).str.strip().str.lower()
             .str.replace("í","i", regex=False)
             .str.replace("á","a", regex=False))
mapping_text2id = {"facil":0, "dificil":1}
y = label_raw.map(mapping_text2id).to_numpy(dtype=int)

X2D = df.drop(columns=[LABEL_COL]).to_numpy(dtype=np.float32)
X = X2D.reshape(len(X2D), 1, X2D.shape[1])  # (n, C=1, L)

strat = y if np.min(np.bincount(y)) >= 2 else None
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=strat
)


mmv.fit_minirocket_parameters(X_train)
Phi_train = mmv.transform_prime(X_train)["phi"]

clf_phi = make_pipeline(
    StandardScaler(with_mean=False),
    LogisticRegression(max_iter=2000, solver="lbfgs", multi_class="auto", n_jobs=-1)
).fit(Phi_train, y_train)
mmv.set_phi_classifier_for_logits(clf_phi)

scaler = clf_phi.named_steps["standardscaler"]
lr     = clf_phi.named_steps["logisticregression"]

rng = np.random.default_rng(42)
bg_size = min(128, len(X_train))
idx_bg = rng.choice(len(X_train), size=bg_size, replace=False)
phi_background = np.vstack([ mmv.transform_prime(X_train[k:k+1])["phi"] for k in idx_bg ])
phi_bg_scaled  = scaler.transform(phi_background)

try:
    explainer = shap.LinearExplainer(lr, phi_bg_scaled)
except Exception:
    explainer = shap.explainers.Linear(lr, phi_bg_scaled)


def _ensure_TC(x):
    x = np.asarray(x)
    if x.ndim == 3:  
        assert x.shape[0] == 1
        x = x[0]
    return x.T if x.shape[0] < x.shape[1] else x

def _flatten_tc(x_tc):
    return _ensure_TC(x_tc).reshape(-1)

def compute_medoids_by_class(X_src, y_src, max_candidates=300, seed=42):
    rng = np.random.default_rng(seed)
    med = {}
    for cls in np.unique(y_src):
        Xc = X_src[y_src == cls]
        if len(Xc) == 0: 
            continue
        idx = np.arange(len(Xc))
        if len(Xc) > max_candidates:
            idx = rng.choice(len(Xc), size=max_candidates, replace=False)
        flat = np.vstack([_flatten_tc(Xc[i:i+1][0].T) for i in idx]).astype(np.float32)
        norms = (flat**2).sum(1, keepdims=True)
        D2 = norms + norms.T - 2.0 * flat @ flat.T
        np.maximum(D2, 0.0, out=D2)
        best = int(np.argmin(D2.mean(1)))
        med[int(cls)] = Xc[idx[best]:idx[best]+1]
    return med

def pick_opposite_medoid(x_raw, clf_phi, medoids, transform_prime):
    phi_x = transform_prime(x_raw)["phi"]
    proba = clf_phi.predict_proba(phi_x)[0]
    classes = clf_phi.classes_
    order = np.argsort(-proba)
    y_hat = int(classes[order[0]])
    for j in order[1:]:
        cand = int(classes[j])
        if cand in medoids:
            return medoids[cand], cand
    for cand in medoids:
        if cand != y_hat:
            return medoids[cand], cand
    raise RuntimeError("No hay medoid opuesto disponible.")


if BASELINE_FROM == "train":
    X_base, y_base = X_train, y_train
else:
    X_base, y_base = X, y
medoids = compute_medoids_by_class(X_base, y_base, max_candidates=300)


if RUN_SCOPE == "test":
    X_run, y_run = X_test, y_test
    split_tag = "test"
else:
    X_run, y_run = X, y
    split_tag = "all"


F = mmv.transform_prime(X_run[0:1])["phi"].shape[1]
T = X_run.shape[2]
N = len(X_run)

betas_path        = os.path.join(OUT_DIR, f"betas_{split_tag}.mmap")
alphas_shap_path  = os.path.join(OUT_DIR, f"alphas_shap_raw_{split_tag}.mmap")
alphas_cal_path   = os.path.join(OUT_DIR, f"alphas_shap_cal_{split_tag}.mmap")

betas_mm       = np.memmap(betas_path,       dtype="float32", mode="w+", shape=(N, T))
alphas_shap_mm = np.memmap(alphas_shap_path, dtype="float32", mode="w+", shape=(N, F))
alphas_cal_mm  = np.memmap(alphas_cal_path,  dtype="float32", mode="w+", shape=(N, F))

rows = []

for i in tqdm(range(N), desc=f"Exportando (SHAP only) [{split_tag}]"):
    x_raw = X_run[i:i+1]
    x_tc  = x_raw[0].T

    # Baseline = medoid opuesto
    X0_op, cls_opp = pick_opposite_medoid(x_raw, clf_phi, medoids, mmv.transform_prime)
    x0_tc = X0_op[0].T

    # Δf (logits)
    fx  = float(mmv.model_logit(x_tc))
    fx0 = float(mmv.model_logit(x0_tc))
    delta_f = fx - fx0

    # ϕ y trazas
    out_x   = mmv.transform_prime(x_raw)
    out_x0  = mmv.transform_prime(X0_op)
    phi_x   = out_x["phi"]
    phi_x0  = out_x0["phi"]
    traces  = out_x["traces"]
    traces0 = out_x0["traces"]

    # α SHAP en ϕ_scaled
    phi_x_scaled  = scaler.transform(phi_x)
    phi_x0_scaled = scaler.transform(phi_x0)

    proba   = clf_phi.predict_proba(phi_x)[0]
    cls_idx = int(np.argmax(proba))

    sv = explainer.shap_values(phi_x_scaled)  # robusto a versiones
    if isinstance(sv, list):
        alphas_shap_raw = sv[cls_idx][0]          # (F,)
    else:
 
        alphas_shap_raw = sv[0] if cls_idx == 1 else -sv[0]


    alphas_shap_cal = mmv.calibrate_alphas_to_delta_f(
        alphas_shap_raw, delta_f, phi_x_scaled, phi_x0_scaled
    )

    sigma_ref = [tr0["sigma"] for tr0 in traces0]
    beta = mmv.propagate_luis(alphas_shap_cal, traces, x_tc, x0_tc, sigma_ref=sigma_ref)

    betas_mm[i, :]       = (beta[:,0] if beta.ndim==2 else beta.ravel()).astype(np.float32)
    alphas_shap_mm[i, :] = alphas_shap_raw.astype(np.float32)
    alphas_cal_mm[i, :]  = alphas_shap_cal.astype(np.float32)

    rows.append({
        "i": i,
        "y_true": int(y_run[i]),
        "y_opp_used": int(cls_opp),
        "delta_f": float(delta_f),
        "sum_alpha_shap_cal": float(np.sum(alphas_shap_cal)),
        "sum_beta": float(beta.sum()),
        "error": float(beta.sum() - delta_f)
    })


del betas_mm, alphas_shap_mm, alphas_cal_mm

df_eff = pd.DataFrame(rows)
df_eff["rel_error"] = (df_eff["error"].abs()/(df_eff["delta_f"].abs()+1e-12))
df_eff.to_csv(os.path.join(OUT_DIR, f"efficiency_{split_tag}.csv"), index=False)

print(f"Listo. Exportado en '{OUT_DIR}':")
print(f"  - {os.path.basename(betas_path)}         (β; shape=({N},{T}))")
print(f"  - {os.path.basename(alphas_shap_path)}   (α SHAP raw; shape=({N},{F}))")
print(f"  - {os.path.basename(alphas_cal_path)}    (α SHAP calibrados; shape=({N},{F}))")
print(f"  - efficiency_{split_tag}.csv")


Exportando (SHAP only) [test]: 100%|██████████| 48/48 [07:22<00:00,  9.22s/it]

Listo. Exportado en 'method1_outputs':
  - betas_test.mmap         (β; shape=(48,7874))
  - alphas_shap_raw_test.mmap   (α SHAP raw; shape=(48,9996))
  - alphas_shap_cal_test.mmap    (α SHAP calibrados; shape=(48,9996))
  - efficiency_test.csv
